In [1]:
outdir = '/home/ben.patterson/projects/simple-pe/simple-pe_tests/h0_peak_params/'

# teobresums environment

In [2]:
import numpy as np
from calcwf import *

In [3]:
# Generate waveform and psd
_, h0, _, _, _, _, _, _, _ = get_h([1,1,1,1], 10, 0.1, chirp2total(24, 2), 2, 4096)
h0 = h0/1000
psd, _ = gen_psd(h0, 10)

# Save waveform and psd
h0.save(outdir+'strain.txt')
psd.save(outdir+'psd.txt')

# simple-pe environment

In [2]:
import numpy as np
import json
from pycbc.types import timeseries, frequencyseries
from simple_pe.param_est import matches

/home/ben.patterson/.conda/envs/igwn_simple_pe_dev/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
# Load data, psd
data = timeseries.load_timeseries(outdir+'strain.txt')
psd = frequencyseries.load_frequencyseries(outdir+'psd.txt')

In [4]:
# Starting params
pars = {'chirp_mass':24, 'symmetric_mass_ratio':2/9, 'chi_align': 0, 'distance': 1}

In [5]:
# Turn off pesummary warnings
import logging
_logger = logging.getLogger('PESummary')
_logger.setLevel(logging.CRITICAL + 10)

In [6]:
# Find peak params
x_max, m = matches.find_best_match(data.real().to_frequencyseries(), pars, ['chirp_mass', 'symmetric_mass_ratio', 'chi_align'], 15, psd)

Calculating the metric | iteration 8 < 20 | error 0.0003 > 0.0003:  


{'chirp_mass': [24], 'symmetric_mass_ratio': [0.2222222222222222], 'chi_align': [0], 'distance': [1]}
idx     chirp_mass     symmetric_mass_ratiochi_align      
0       -0.000370      0.008961       0.002430       



TypeError: 'float' object is not iterable